## Methods & Settings

In [1]:
from IPython.display import HTML
from IPython.display import display

# Taken from https://stackoverflow.com/questions/31517194/how-to-hide-one-specific-cell-input-or-output-in-ipython-notebook
tag = HTML('''<script>
code_show=false; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<a href="javascript:code_toggle()">HIDE/SHOW CONTENT</a>.''')
display(tag)

############### Write code below ##################

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import gc
import math

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:75% !important; }</style>"))

pd.set_option('display.max_rows', 250)
pd.set_option('display.min_rows', 25)
pd.set_option('display.max_columns', 50)

####
# prints memory usage
def show_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB\n'.format(start_mem))
    return

####
# seperates features from label (y must be last column)
def sep_X_y(df):
    X = df.iloc[:,0:-1] # extracts all rows [:] and columns from 0 to next-to-last [0:-1]
    y = df.iloc[:,-1] # extracts all rows [:] and only last column [-1]
    
    return [X, y]

####
# split training and test set from given dataframe with month as boundaries
def mth_train_test_split(df, mth_start, mth_end):
    print('Splitting dataframe...\n')
    
    # get indices from desired boundaries
    idx_start = df.month.searchsorted(mth_start_train, side='left') # list needs to be sorted already for searchsorted
    idx_end = df.month.searchsorted(mth_end_train + 1, side='left')
    
    df = df.iloc[idx_start:idx_end]
    
    return df

####
# trains LinearRegression model
def train_linReg(X, y):
    
    print('Fitting model...')
    model = LinearRegression()
    fitted_model = model.fit(X, y)
    print('Done!')
    
    return fitted_model

####
# predicts labels of training and test with given model
def predict_values(model, X, y_true):
    print('Predicting values...')
    # predict y values
    y_pred = model.predict(X)
    print('Done!\n')
    
    # get msq
    model_error = mean_squared_error(y_true, y_pred)
    
    # print info about accuracies
    print(f'\t\t\t\t\t\033[1m LinearRegression MSE: '
          f' {model_error:.3f}')
    
    # return predicted values
    return y_pred

####
# concatenates prediction with actual target for evaluation
def concat_ytrue_ypred(X, y_true, y_pred):
    # create dataframe from test-prediction with index from X_test
    df_y_pred = pd.DataFrame(y_pred, columns=['nextBuyIn_pred'], index=X.index, dtype=np.int8)

    # concatenate X, y, y_pred (put columns next to each other)
    df_eval = pd.concat([X, y_true, df_y_pred], axis=1)
    
    return df_eval

# <font color='Black'>Predicting Train/Test Split</color>


### Read data

In [2]:
train = r'E:\OneDrive\Arbeit\Repos\DMC2022\Kevin\csv\17-1_220624_4TimeRepurchaser_train.csv'
test = r'E:\OneDrive\Arbeit\Repos\DMC2022\Kevin\csv\17-2_220624_4TimeRepurchaser_test.csv'

columns = [#'date',
           'userID', 
           'itemID',
           'order', 
           'brand', 
           'feature_1', 
           'feature_2', 
           'feature_3', 
           'feature_4', 
           'feature_5',
           'categories',
           'brandOrderRatio',
           'feature1OrderRatio',
           'feature2OrderRatio',
           'feature3OrderRatio',
           'feature4OrderRatio',
           'feature5OrderRatio',
           'TotalBFscore',
           'RCP',
           'MeanDiffToNxt(user)',
           'TotalItemOrders(user)',
           #'TotalItemOrders(item)',
           'date(year)',
           'date(month)',
           #'date(weekOfMonth)',
           'date(dayOfMonth)',
           'date(weekOfYear)',
           'date(dayOfYear)',
           #'nextBuyInWeeks(round)', # label
           'nextBuyInWeeks(floor)', # label
           #'nextBuyInWeekOfYear' # label; schlechte idee
          ]

dtype = {'userID':np.uint16,
         'itemID':np.uint16,
         'order':np.uint8,
         'brand':np.int16,
         'feature_1':np.int8,
         'feature_2':np.uint8,
         'feature_3':np.int16,
         'feature_4':np.int8,
         'feature_5':np.int16,
         'TotalItemOrders(user)':np.uint16,
         'date(year)':np.uint16,
         'date(month)':np.uint8,
         'date(weekOfMonth)':np.uint8,
         'date(dayOfMonth)':np.uint8,
         'date(weekOfYear)':np.uint8,
         'date(dayOfYear)':np.uint16,
         'nextBuyInWeeks(floor)':np.uint8
        }

label = 'nextBuyInWeeks(floor)'

df_train = pd.read_csv(train, sep='|', usecols=columns, dtype=dtype, nrows=None, converters={
    'categories': lambda x: [int(i) for i in x[1:-1].split(',')]
})

df_test = pd.read_csv(test, sep='|', usecols=columns, dtype=dtype, nrows=None, converters={
    'categories': lambda x: [int(i) for i in x[1:-1].split(',')]
})

## Data preparation

### Add fake column to ensure all categories are included in Multi-Hot-Encoding

In [3]:
# add fake column for ensuring all categories from 0 to 4299 are included
df_train.loc[len(df_train)] = [0 if column != 'categories' else [cat for cat in range(0,4300)] for column in df_train.columns]
df_train.index = df_train.index + 1  # add index

df_test.loc[len(df_test)] = [0 if column != 'categories' else [cat for cat in range(0,4300)] for column in df_test.columns]
df_test.index = df_test.index + 1  # add index

df = df_train

### Multi-Hot-Encoding

#### Trainingset

In [4]:
# multi-hot-encode categories
cats = df["categories"]
mlb = MultiLabelBinarizer(sparse_output=False) # Set to True if output binary array is desired in CSR sparse format
df_multi_hot = pd.DataFrame(mlb.fit_transform(cats), columns=mlb.classes_, index=df.index, dtype=np.int8).astype(pd.SparseDtype(np.uint8,0)) # NaN filled with 0

# drop fake rows from both dataframes (last row) & drop category '9999' standing for missing category
df_multi_hot.drop(index=df.index[-1], axis=0, inplace=True)
df_multi_hot = df_multi_hot.iloc[:,:-1]
df.drop(index=df.index[-1], axis=0, inplace=True)

# join new binarized columns with rest of dataframe
df = df.join(df_multi_hot, how='inner')

if (len(df[df.isnull().any(axis=1)]) > 0):
    raise RuntimeError('Join of multi-hot-encoded categories probably created missing values.')

# drop list of categories, since it's not needed anymore
df.drop('categories', axis=1, inplace=True)

# pop and append 'week' at end of dataframe
col = df.pop(label)
df.insert(len(df.columns), col.name, col)

del df_multi_hot
gc.collect()

df

userID  itemID  order  brand  feature_1  feature_2  feature_3  \
1         276   15667      1   1201          4          0         30   
2         276   28708      1    504         10          0        441   
3         532    7644      1   1276          6          0         45   
4         752   22963      1   1201         10          0         43   
5        1123   18498      1   1401          4          0         95   
6        1421   20664      1    408          4          0        284   
7        1524   20703      1    745         10          0        503   
8        1524   22341      1    386         10          0        502   
9        1524   23688      3    420          4          1        510   
10       1567   29726      1    378         10          0        421   
11       1655   12379      1    484         10          0        503   
12       1662   29132      2    325          4          0        487   
...       ...     ...    ...    ...        ...        ...        ...   
39725   39937   22631      4    120          4          0        396   
39726    8786   14420      1    703          4          0        335   
39727   36093   17230      4   1163          4          0        448   
39728   36093   18340      5   1163          4          1        448   
39729   31993    3614      2   1111          6          0         41   
39730   22815   10849      4    827         10          0        348   
39731    6457   30362      5    618          4          0         21   
39732    8786   14420      1    703          4          0        335   
39733   19221    8927      2    745         10          0        503   
39734   34638    4935      1   1127          4          0        360   
39735   21517     594      1    203          4          1        491   
39736   21517   19443      1    408         10          0        160   

       feature_4  feature_5  brandOrderRatio  feature1OrderRatio  \
1              0        163         0.000659            0.023826   
2              3         84         0.000978            0.020648   
3              3         48         0.000081            0.005880   
4              0        147         0.000659            0.020648   
5              0         44         0.000074            0.023826   
6              0         66         0.000455            0.023826   
7              0         17         0.000519            0.020648   
8              0         29         0.000194            0.020648   
9              0         -1         0.000007            0.023826   
10             0          3         0.000396            0.020648   
11             0         17         0.000087            0.020648   
12             3         37         0.000605            0.023826   
...          ...        ...              ...                 ...   
39725          0        144         0.000080            0.023826   
39726          0        132         0.001555            0.023826   
39727          0        161         0.000283            0.023826   
39728          0        161         0.000283            0.023826   
39729          0        146         0.000753            0.005880   
39730          0         95         0.000829            0.020648   
39731          3         52         0.001137            0.023826   
39732          0        132         0.001555            0.023826   
39733          0         85         0.000519            0.020648   
39734          3        144         0.000231            0.023826   
39735          0         66         0.000049            0.023826   
39736          0         38         0.000455            0.020648   

       feature2OrderRatio  feature3OrderRatio  feature4OrderRatio  \
1                0.044684            0.000067            0.033429   
2                0.044684            0.001161            0.016194   
3                0.044684            0.000031            0.016194   
4                0.044684            0.000451            0.033429   
5                0.0

#### Testset

In [5]:
# multi-hot-encode categories
cats = df_test["categories"]
mlb = MultiLabelBinarizer(sparse_output=False) # Set to True if output binary array is desired in CSR sparse format
df_multi_hot = pd.DataFrame(mlb.fit_transform(cats), columns=mlb.classes_, index=df_test.index, dtype=np.int8).astype(pd.SparseDtype(np.uint8,0)) # NaN filled with 0

# drop fake rows from both dataframes (last row) & drop category '9999' standing for missing category
df_multi_hot.drop(index=df_test.index[-1], axis=0, inplace=True)
df_multi_hot = df_multi_hot.iloc[:,:-1]
df_test.drop(index=df_test.index[-1], axis=0, inplace=True)

# join new binarized columns with rest of dataframe
df_test = df_test.join(df_multi_hot, how='inner')

if (len(df_test[df_test.isnull().any(axis=1)]) > 0):
    raise RuntimeError('Join of multi-hot-encoded categories probably created missing values.')

# drop list of categories, since it's not needed anymore
df_test.drop('categories', axis=1, inplace=True)

# pop and append 'week' at end of dataframe
col = df_test.pop(label)
df_test.insert(len(df_test.columns), col.name, col)

del df_multi_hot
gc.collect()

0

### Split Train/Test

In [6]:
# split df in X & y

X_train, y_train = sep_X_y(df)
X_test, y_test = sep_X_y(df_test)

## Training & Prediction with Linear Regression

In [7]:
# Pipeline needs training method, dataframe and dates to split dataframe in training and test set.
model = train_linReg(X_train, y_train)

Fitting model...
Done!


## Evaluation

### Trainingset eval

In [8]:
y_pred = predict_values(model, X_train, y_train)

Predicting values...
Done!

					 LinearRegression MSE:  7.520


In [9]:
# Change dtypes to save RAM
dtype_X = {'userID':np.uint16,
         'itemID':np.uint16,
         'order':np.uint8,
         'brand':np.int16,
         'feature_1':np.int8,
         'feature_2':np.uint8,
         'feature_3':np.int16,
         'feature_4':np.int8,
         'feature_5':np.int16,
         'TotalItemOrders(user)':np.uint16,
         'date(year)':np.uint16,
         'date(month)':np.uint8,
         #'date(weekOfMonth)':np.uint8,
         'date(dayOfMonth)':np.uint8,
         'date(weekOfYear)':np.uint8,
         'date(dayOfYear)':np.uint16
        }

dtype_y = {'nextBuyInWeeks(floor)':np.uint8}

y_pred = pd.DataFrame(y_pred, index=y_train.index).apply(lambda x: round(x)).astype(np.uint8)

y_pred.set_axis(['nextBuyIn_pred'], axis=1,inplace=True)

# concatenate X, y, y_pred (columns next to each other)
df_eval = pd.concat([X_train, y_train, y_pred], axis=1)

rowcount = len(df_eval)
should = rowcount
is_ = len(df_eval.loc[(df_eval['nextBuyInWeeks(floor)'] == df_eval.nextBuyIn_pred)]) 

print(f'\033[1mrow count of set:\t\t\t\t {rowcount}')
print(f'\033[1mrows where label was predicted correctly:\t {is_} \t ({is_/should*100:.3f} % of rows)')

df_eval

row count of set:				 39736
rows where label was predicted correctly:	 7420 	 (18.673 % of rows)


userID  itemID  order  brand  feature_1  feature_2  feature_3  \
1         276   15667      1   1201          4          0         30   
2         276   28708      1    504         10          0        441   
3         532    7644      1   1276          6          0         45   
4         752   22963      1   1201         10          0         43   
5        1123   18498      1   1401          4          0         95   
6        1421   20664      1    408          4          0        284   
7        1524   20703      1    745         10          0        503   
8        1524   22341      1    386         10          0        502   
9        1524   23688      3    420          4          1        510   
10       1567   29726      1    378         10          0        421   
11       1655   12379      1    484         10          0        503   
12       1662   29132      2    325          4          0        487   
...       ...     ...    ...    ...        ...        ...        ...   
39725   39937   22631      4    120          4          0        396   
39726    8786   14420      1    703          4          0        335   
39727   36093   17230      4   1163          4          0        448   
39728   36093   18340      5   1163          4          1        448   
39729   31993    3614      2   1111          6          0         41   
39730   22815   10849      4    827         10          0        348   
39731    6457   30362      5    618          4          0         21   
39732    8786   14420      1    703          4          0        335   
39733   19221    8927      2    745         10          0        503   
39734   34638    4935      1   1127          4          0        360   
39735   21517     594      1    203          4          1        491   
39736   21517   19443      1    408         10          0        160   

       feature_4  feature_5  brandOrderRatio  feature1OrderRatio  \
1              0        163         0.000659            0.023826   
2              3         84         0.000978            0.020648   
3              3         48         0.000081            0.005880   
4              0        147         0.000659            0.020648   
5              0         44         0.000074            0.023826   
6              0         66         0.000455            0.023826   
7              0         17         0.000519            0.020648   
8              0         29         0.000194            0.020648   
9              0         -1         0.000007            0.023826   
10             0          3         0.000396            0.020648   
11             0         17         0.000087            0.020648   
12             3         37         0.000605            0.023826   
...          ...        ...              ...                 ...   
39725          0        144         0.000080            0.023826   
39726          0        132         0.001555            0.023826   
39727          0        161         0.000283            0.023826   
39728          0        161         0.000283            0.023826   
39729          0        146         0.000753            0.005880   
39730          0         95         0.000829            0.020648   
39731          3         52         0.001137            0.023826   
39732          0        132         0.001555            0.023826   
39733          0         85         0.000519            0.020648   
39734          3        144         0.000231            0.023826   
39735          0         66         0.000049            0.023826   
39736          0         38         0.000455            0.020648   

       feature2OrderRatio  feature3OrderRatio  feature4OrderRatio  \
1                0.044684            0.000067            0.033429   
2                0.044684            0.001161            0.016194   
3                0.044684            0.000031            0.016194   
4                0.044684            0.000451            0.033429   
5                0.0

### Testset eval

In [10]:
y_pred = predict_values(model, X_test, y_test)

Predicting values...
Done!

					 LinearRegression MSE:  16.030


In [11]:
dtype_X = {'userID':np.uint16,
         'itemID':np.uint16,
         'order':np.uint8,
         'brand':np.int16,
         'feature_1':np.int8,
         'feature_2':np.uint8,
         'feature_3':np.int16,
         'feature_4':np.int8,
         'feature_5':np.int16,
         'TotalItemOrders(user)':np.uint16,
         'date(year)':np.uint16,
         'date(month)':np.uint8,
         #'date(weekOfMonth)':np.uint8,
         'date(dayOfMonth)':np.uint8,
         'date(weekOfYear)':np.uint8,
         'date(dayOfYear)':np.uint16
        }
dtype_y = {'nextBuyInWeeks(floor)':np.uint8}

y_pred = pd.DataFrame(y_pred, index=y_test.index).apply(lambda x: round(x)).astype(np.uint8)

y_pred.set_axis(['nextBuyIn_pred'], axis=1,inplace=True)
# concatenate X, y, y_pred (columns next to each other)
df_eval = pd.concat([X_test, y_test, y_pred], axis=1)

rowcount = len(df_eval)
should = rowcount
is_ = len(df_eval.loc[(df_eval['nextBuyInWeeks(floor)'] == df_eval.nextBuyIn_pred)]) 

print(f'\033[1mrow count of set:\t\t\t\t {rowcount}')
print(f'\033[1mrows where label was predicted correctly:\t {is_} \t ({is_/should*100:.3f} % of rows)')

row count of set:				 13591
rows where label was predicted correctly:	 1982 	 (14.583 % of rows)


---

# Comparison with baseline model: Adding mean time between purchases to last purchase date

In [12]:
test = r'E:\OneDrive\Arbeit\Repos\DMC2022\Kevin\csv\17-2_220624_4TimeRepurchaser_test.csv'

columns = ['date',
           'userID', 
           'itemID',
           'order', 
           'brand', 
           'feature_1', 
           'feature_2', 
           'feature_3', 
           'feature_4', 
           'feature_5',
           'categories',
           'brandOrderRatio',
           'feature1OrderRatio',
           'feature2OrderRatio',
           'feature3OrderRatio',
           'feature4OrderRatio',
           'feature5OrderRatio',
           'TotalBFscore',
           'RCP',
           'MeanDiffToNxt(user)',
           'TotalItemOrders(user)',
           #'TotalItemOrders(item)',
           'date(year)',
           'date(month)',
           #'date(weekOfMonth)',
           'date(dayOfMonth)',
           'date(weekOfYear)',
           'date(dayOfYear)',
           #'nextBuyInWeeks(round)', # label
           'nextBuyInWeeks(floor)', # label
           #'nextBuyInWeekOfYear' # label; schlechte idee
          ]

dtype = {'userID':np.uint16,
         'itemID':np.uint16,
         'order':np.uint8,
         'brand':np.int16,
         'feature_1':np.int8,
         'feature_2':np.uint8,
         'feature_3':np.int16,
         'feature_4':np.int8,
         'feature_5':np.int16,
         'TotalItemOrders(user)':np.uint16,
         'date(year)':np.uint16,
         'date(month)':np.uint8,
         'date(weekOfMonth)':np.uint8,
         'date(dayOfMonth)':np.uint8,
         'date(weekOfYear)':np.uint8,
         'date(dayOfYear)':np.uint16,
         'nextBuyInWeeks(floor)':np.uint8
        }

label = 'nextBuyInWeeks(floor)'

In [13]:
df_test = pd.read_csv(test, sep='|', usecols=columns, dtype=dtype, nrows=None, converters={
    'categories': lambda x: [int(i) for i in x[1:-1].split(',')]
})

In [14]:
df_test['meanPred'] = (df_test['date(weekOfYear)'] + round(df_test['MeanDiffToNxt(user)']/7)).astype(np.uint16)
df_test['y_true'] = df_test['date(weekOfYear)'] + df_test['nextBuyInWeeks(floor)']

rowcount = len(df_test)
should = rowcount
is_ = len(df_test.loc[(df_test['meanPred'] == df_test.y_true)]) 

print(f'\033[1mrow count of set:\t\t\t\t {rowcount}')
print(f'\033[1mrows where label was predicted correctly:\t {is_} \t ({is_/should*100:.3f} % of rows)')

row count of set:				 13591
rows where label was predicted correctly:	 1607 	 (11.824 % of rows)


---

# Baseline model:                   11.8% of rows correct
# Linear Regression model:  14.6% of rows correct